# Image_Classification_Pytorch (Training)
https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html

https://www.kaggle.com/zzy990106/pytorch-5-fold-efficientnet-baseline

<a class="anchor" id="0"></a>
# Table of Contents

1. [套件安裝與載入](#1)
1. [環境檢測與設定](#2)
1. [開發參數設定](#3)
1. [資料處理](#4)
    -  [載入CSV檔](#4.1)
    -  [檢查CSV檔缺失值](#4.2)
1. [定義模型方法](#5)
1. [製作資料集＆資料擴增&回調函數&訓練模型](#6)
1. [混淆矩陣](#7)
1. [待辦事項](#8)

# 1. 套件安裝與載入<a class="anchor" id="1"></a>
[Back to Table of Contents](#0)

In [ ]:
# 資料處理套件
import os
import gc
import sys
import time
import copy
import random
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import albumentations as A
import matplotlib.pyplot as plt

from tqdm import tqdm
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, StratifiedKFold
from albumentations.pytorch.transforms import ToTensorV2

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# 設定顯示中文字體
from matplotlib.font_manager import FontProperties
plt.rcParams['font.sans-serif'] = ['Microsoft JhengHei'] # 用來正常顯示中文標籤
plt.rcParams['font.family'] = 'AR PL UMing CN'
plt.rcParams['axes.unicode_minus'] = False # 用來正常顯示負號

In [ ]:
# pytorch深度學習模組套件
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torch.nn.functional as F
import torchvision

from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
from torchvision import models

# 2. 環境檢測與設定<a class="anchor" id="2"></a>
[Back to Table of Contents](#0)

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE

In [ ]:
'''執行環境參數設定'''

# (Boolean)是否為本機
LOCAL = False

# (Boolean)是否為 Colab
COLAB = False


'''檔案路徑參數設定'''

# (String)Root路徑
if LOCAL:
    PATH = r'../'
elif COLAB:
    PATH = r'/content/drive/My Drive/Colab Notebooks/'
else:
    PATH = r'../input/'
    OUTPUT_PATH = r'/kaggle/working/'
    
# (String)資料根路徑
DATA_ROOT_PATH = PATH+r'datasets/AI_CUP_2020_AIMango_Defective_Classification/' 

# (String)訓練資料路徑
TRAIN_DATA_PATH = DATA_ROOT_PATH+r'C1-P2_Train Dev/Train'

# (String)訓練CSV路徑，如為None則不讀CSV檔
TRAIN_CSV_PATH = DATA_ROOT_PATH+r'C1-P2_Train Dev/train.csv'

# (String)專案名稱
PROJECT_NAME = 'AI_CUP_2020_AIMango_Defective_Classification'

# (String)專案檔案儲存路徑
if LOCAL or COLAB:
    OUTPUT_PATH = PATH
PROJECT_PATH = OUTPUT_PATH+PROJECT_NAME+'/'+PROJECT_NAME+' '+datetime.datetime.now().strftime("%Y-%m-%d %H:%M")

# (String)權重名稱(使用哪個權重)
WEIGHTS_NAME = 'efficientnetb7'

# (String)模型名稱(使用哪個模型)
MODEL_NAME = 'efficientnetb7'

# (String)讀取預訓練權重的儲存路徑 
LOAD_WEIGHTS_PATH = PROJECT_PATH+r'/models/backup/'+WEIGHTS_NAME+'.pth'

# (String)讀取預訓練模型的儲存路徑 
LOAD_MODEL_PATH = PROJECT_PATH+r'/models/backup/'+MODEL_NAME+'.pth'

# (String)訓練模型的儲存路徑
TRAIN_MODEL_PATH = PROJECT_PATH+r'/models/'+MODEL_NAME+'.pth'

In [ ]:
if not LOCAL and COLAB:
    from google.colab import drive
    drive.mount('/content/drive')

In [ ]:
if DEVICE != "CPU":
    !nvidia-smi

In [ ]:
if os.path.isfile(TRAIN_CSV_PATH):
    LOAD_CSV = True
else:
    LOAD_CSV = False

In [ ]:
if not os.path.isdir(PROJECT_PATH+r'/models/'):
    os.makedirs(PROJECT_PATH+r'/models/')

# 3. 開發參數設定<a class="anchor" id="3"></a>
[Back to Table of Contents](#0)

In [ ]:
'''客製參數設定'''


'''資料參數設定'''

# (Int)分類數量
CLASSES = 2

# (Int)有CSV檔該參數才有用，1則為不做交叉驗證
FOLD = 1

# (Int)沒CSV檔，FOLD該參數固定為1
if not LOAD_CSV:
    FOLD = 1
    
# (Int)圖片尺寸
IMAGE_SIZE = [224]*FOLD

# (String)圖片副檔名
IMAGE_NAME_EXTENSION = '.jpg'

# (String)CSV圖片檔名欄位(不包含路徑)
IMAGE_NAME = 'image_id'

# (String)CSV圖片檔名欄位(包含路徑)
IMAGE_NAME_ROOT = 'image'

# (String)CSV標籤欄位
LABEL_NAME = 'grade'

# (String)CSV標籤欄位類型
LABEL_NAME_TYPE = 'string'

# (Boolean)CSV圖片檔名欄位是否包含副檔名
IMAGE_NAME_HAVE_EXTENSION = True

# (Int)不包含副檔名的圖片檔名長度，因為CSV檔名欄位有副檔名時需要移除
IMAGE_NAME_LENGTH = 5

# (Int)不同的種子會產生不同的Random或分層K-FOLD分裂, 42則是預設固定種子
SEED = 42

if FOLD == 1:
    # (Float)驗證集佔訓練集的比率，FOLD>1則不啟用
    DATA_SPLIT = 0.2
else:
    # (String)切分訓練集跟驗證集方式
    SKF = StratifiedKFold(n_splits=FOLD,shuffle=True,random_state=SEED)

# (Boolean)如為True每次返回的卷積算法將是確定的，即默認算法
CUDNN_DETERMINISTIC = True

# (Boolean)PyTorch 中對模型裡的卷積層進行預先的優化，也就是在每一個卷積層中測試 cuDNN 提供的所有卷積實現算法，
# 然後選擇最快的那個。這樣在模型啟動的時候，只要額外多花一點點預處理時間，就可以較大幅度地減少訓練時間
CUDNN_BENCHMARK = True


'''資料擴增參數設定

資料擴增範例
https://zh-hant.hotbak.net/key/albumentation%E6%95%B8%E6%93%9A%E5%A2%9E%E5%BC%B7CSDN.html

資料擴增教學
https://zhuanlan.zhihu.com/p/107399127

資料擴增Doc
https://vfdev-5-albumentations.readthedocs.io/en/docs_pytorch_fix/api/augmentations.html
'''

# (Float)訓練集資料擴增的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_TRAIN_TRANSFORMS = 1.0

# (Float)驗證集資料擴增的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_VAL_TRANSFORMS = 1.0

# 以下資料擴增為訓練集使用=============================================

# (Float)模糊的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_BLUR = 0

# (Int)模糊的上限
BLUR_LIMIT = 3

# (Float)水平翻轉的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_HORIZONTALFLIP = 0.5

# (Float)垂直翻轉的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_VERTICALFLIP = 0

# (Float)水平和垂直翻轉的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_FLIP = 0

# (Float)隨機旋轉90度的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_RANDOMROTATE90 = 0

# (Float)平移縮放旋轉的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_SHIFTSCALEROTATE = 0

# (Float)平移縮放旋轉的平移上限
SHIFTSCALEROTATE_SHIFT_LIMIT = 0.0625

# (Float)平移縮放旋轉的縮放上限
SHIFTSCALEROTATE_SCALE_LIMIT = 0.1

# (Float)平移縮放旋轉的旋轉上限
SHIFTSCALEROTATE_SHIFT_LIMIT = 45

# (Float)彈性變換的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_ELATICTRANSFORM = 0

# (Float)彈性變換的alpha高斯過濾參數
ELATICTRANSFORM_ALPHA = 1

# (Float)彈性變換的sigma高斯過濾參數
ELATICTRANSFORM_SIGMA = 50

# (Float)彈性變換的alpha_affine，範圍為（-alpha_affine，alpha_affine）
ELATICTRANSFORM_ALPHA_AFFINE = 50

# (Float)網格失真的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_GRIDDISTORTION = 0

# (Int)網格失真的每一條邊上網格單元數量
GRIDDISTORTION_NUM_STEPS = 5

# (Float)隨機亮度對比度的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_RANDOMBRIGHTNESSCONTRAST_CONTRAST = 0

# (Float)隨機亮度的上限
RANDOMBRIGHTNESSCONTRAST_BRIGHTNESS_LIMIT = 0.2

# (Float)隨機對比度的上限
RANDOMBRIGHTNESSCONTRAST_CONTRAST_LIMIT = 0.2

# (Float)隨機色調飽和度的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_HUESATURATIONVALUE = 0

# (Float)隨機色調飽和度的色調上限
HUESATURATIONVALUE_HUE_SHIFT_LIMIT = 20

# (Float)隨機色調飽和度的飽和度上限
HUESATURATIONVALUE_SAT_SHIFT_LIMIT = 30

# (Float)隨機色調飽和度的值上限
HUESATURATIONVALUE_VAL_SHIFT_LIMIT = 20

# (Float)對比度受限自適應直方圖均衡的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_CLAHE = 0

# (Float)對比度受限自適應直方圖均衡的對比度上限
CLAHE_CLIP_LIMIT = 4.0

# (Float)隨機在圖像上生成黑色矩形的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_COARSEDROPOUT = 0

# (Int)隨機在圖像上生成黑色矩形的數量
COARSEDROPOUT_NUM_HOLES = 0

# (Int)隨機在圖像上生成黑色矩形的最大高度
COARSEDROPOUT_MAX_H_SIZE = 8

# (Int)隨機在圖像上生成黑色矩形的最大寬度
COARSEDROPOUT_MAX_W_SIZE = 8

# (Float)隨機縮放剪裁的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_RANDOMRESIZEDCROP = 0

# (Float Tuple)隨機縮放剪裁之前的圖像比例縮放
RANDOMRESIZEDCROP_SCALE = (0.08, 1.0)

# (Int)隨機縮放剪裁之前的圖像高度
RANDOMRESIZEDCROP_HEIGHT = IMAGE_SIZE[0]

# (Int)隨機縮放剪裁之前的圖像寬度
RANDOMRESIZEDCROP_WIDTH = IMAGE_SIZE[0]

# 以上資料擴增為訓練集使用=============================================

# 以下資料擴增為訓練集和驗證集共用======================================

# (Float)縮放的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_RESIZE = 1.0

# (Int)縮放後的圖片高度
RESIZE_HEIGHT = IMAGE_SIZE[0]

# (Int)縮放後的圖片寬度
RESIZE_WIDTH = IMAGE_SIZE[0]

# (Float)正規化的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
P_NORMALIZE = 1.0

# (List)正規化的平均值(Imagenet的參考平均值[0.485, 0.456, 0.406])
NORMALIZE_MEAN = [0.485, 0.456, 0.406]

# (List)正規化的標準差(Imagenet的參考標準差[0.229, 0.224, 0.225])
NORMALIZE_STD = [0.229, 0.224, 0.225]

# (Float)正規化的PIXEL最大值(Imagenet的參考PIXEL最大值255.0)
NORMALIZE_MAX_PIXEL_VALUE = 255.0

# (Float)歸一化的啟用(0:不啟用,1.0:一律啟用,小數點:機率啟用)
# ToTensorV2()將[0, 255]的PIL.Image或[H, W, C]的numpy.ndarray數據，
# 轉換為形狀[C, H, W]的torch.FloadTensor，並歸一化到[0, 1.0]。
P_TOTENSORV2 = 1.0

# 以上資料擴增為訓練集和驗證集共用======================================


''''模型參數設定'''

# (Boolean)使用Pytorch模型，如為False則須客制另外撰寫
USE_BASE_MODEL = False

if USE_BASE_MODEL:
    # (Model)建立Pytorch模型
    BASE_MODEL = models.resnet18
    
# (Boolean)是否使用Pytorch權重
LOAD_PYTORCH_WEIGHTS = True

# (Boolean)Pytorch模型是否包含完全連接網路頂部的網路層
INCLUDE_TOP = True

# (Boolean)Pytorch模型是否可訓練權重(不包括頂部網路層)
BASE_MODEL_TRAINABLE = True

# (Boolean)是否已有客製模型，僅載入權重
LOAD_WEIGHTS = False

# (Boolean)是否載入完整客製(模型+權重)
LOAD_MODEL = False

# (Boolean)是否印出完整模型
MODEL_PRINT = True


''''回調函數參數設定

學習率遞減
https://zhuanlan.zhihu.com/p/69411064

模型儲存
https://pytorch.org/tutorials/beginner/saving_loading_models.html

'''

# (Boolean)回調函數 ModelCheckpoint 是否啟用
CALLBACKS_CHECK_POINTER = True

# (String)回調函數監控數值(val_auc 僅限雙分類) val_acc/val_loss/val_auc
MONITOR = 'val_loss'

# (Boolean)回調函數 ModelCheckpoint 是否只儲存最佳模型 False
SAVE_BEST_ONLY = True

# (Boolean)回調函數 ModelCheckpoint 是否只儲存權重 True
SAVE_WEIGHTS_ONLY = True


''''編譯參數設定'''

# (Boolean)是否印出完整編譯器
OPTIMIZER_PRINT = True

# (Float)優化器學習率 1e-3/1e-1
LEARNING_RATE = 1e-3

# (Float)優化器權重衰減 5e-5/5e-4
WEIGHT_DECAY = 5e-5

# (Float)加速優化器在相關方向上前進，並抑制震盪 0.9
MOMENTUM = None

# (String)優化器指定，None為客制，須另外撰寫
BASE_OPTIMIZERS = optim.Adam

# (String)損失函數，None為客制，須另外撰寫
BASE_LOSSES = nn.CrossEntropyLoss

# (String)指定還原成適用於輸出，預設mean
REDUCTION = "mean"


''''訓練參數設定'''

# (Int List)每批訓練的尺寸
BATCH_SIZE = [16]*FOLD

# (Int)訓練做幾次時代
EPOCHS = [2]*FOLD

# (Int)要用於數據加載的子進程數。0表示將在主進程中加載數據。（默認值：0）
NUM_WORKERS = 0

# (Boolean)批次處理在大小不合適的情況下，是否刪除最後一個不完整的批次
DROP_LAST = False

# (Int)指定列印進度條的位置（從0開始）。
TQDM_POSITION = 0

# (Boolean)保留迭代結束時進度條的所有痕跡。如果是None，只會在position是0時離開
TQDM_LEAVE = True


''''圖表參數設定'''

# (Float)全部SNS圖表的字形縮放
ALL_SNS_FONT_SCALE = 1.0

# (Int)CSV缺失值圖表寬度
CSV_COUNTPLOT_FIGSIZE_W = 10

# (Int)CSV缺失值圖表高度
CSV_COUNTPLOT_FIGSIZE_H = 10

# (Int)CSV缺失值圖表標題字型大小
CSV_COUNTPLOT_TITLE_FONTSIZE = 20

# (Int)CSV缺失值圖表X軸標題字型大小
CSV_COUNTPLOT_XLABEL_FONTSIZE = 15

# (Int)CSV缺失值圖表Y軸標題字型大小
CSV_COUNTPLOT_YLABEL_FONTSIZE = 15

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = CUDNN_DETERMINISTIC
    torch.backends.cudnn.benchmark = CUDNN_BENCHMARK

seed_everything(SEED)

In [ ]:
# 設置sns圖表縮放係數
sns.set(font_scale = ALL_SNS_FONT_SCALE)

# 4. 資料處理<a class="anchor" id="4"></a>
[Back to Table of Contents](#0)

## 4.1 載入CSV檔 <a class="anchor" id="4.1"></a>
[Back to Table of Contents](#0)

In [ ]:
if LOAD_CSV:
    print('Reading data...')

    # 讀取訓練資料集CSV檔
    train_csv = pd.read_csv(TRAIN_CSV_PATH,encoding="utf8")

    print('Reading data completed')

In [ ]:
if LOAD_CSV:
    # CSV檔名欄位包括副檔名時，需要移除副檔名
    if IMAGE_NAME_HAVE_EXTENSION:
        train_csv[IMAGE_NAME] = train_csv[IMAGE_NAME].str.slice(stop = IMAGE_NAME_LENGTH)

In [ ]:
if LOAD_CSV:
    # 顯示訓練資料集CSV檔
    print(train_csv.head())

In [ ]:
if LOAD_CSV:
    print("Shape of train_data :", train_csv.shape)

## 4.2 檢查CSV檔缺失值 <a class="anchor" id="4.2"></a>
[Back to Table of Contents](#0)

In [ ]:
if LOAD_CSV:
    total = train_csv.isnull().sum().sort_values(ascending = False)
    percent = (train_csv.isnull().sum()/train_csv.isnull().count()*100).sort_values(ascending = False)
    missing_train_csv  = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
    print(missing_train_csv.head())

In [ ]:
if LOAD_CSV:
    print(train_csv[LABEL_NAME].value_counts())
    f,ax = plt.subplots(figsize=(CSV_COUNTPLOT_FIGSIZE_W, CSV_COUNTPLOT_FIGSIZE_H))
    sns.countplot(train_csv[LABEL_NAME], hue = train_csv[LABEL_NAME],ax = ax)
    plt.title("LABEL COUNT", fontsize=CSV_COUNTPLOT_TITLE_FONTSIZE)
    plt.xlabel(LABEL_NAME.upper(), fontsize=CSV_COUNTPLOT_XLABEL_FONTSIZE)
    plt.ylabel("COUNT", fontsize=CSV_COUNTPLOT_YLABEL_FONTSIZE)
    plt.legend()
    plt.show()

# 5. 定義模型方法<a class="anchor" id="5"></a>
[Back to Table of Contents](#0)

In [ ]:
def build_optimizers():
    if BASE_OPTIMIZERS == None:
        print("Custiom OPTIMIZERS")
    else:
        RETURN_OPTIMIZERS = BASE_OPTIMIZERS
    return RETURN_OPTIMIZERS

optimizer = build_optimizers()

In [ ]:
def build_losses():
    if BASE_LOSSES == None:
        print("Custiom LOSSES")
    else:
        RETURN_LOSSES = BASE_LOSSES
    return RETURN_LOSSES

loss = build_losses()

In [ ]:
class build_model(nn.Module):

    def __init__(self):
        super(build_model, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
if LOAD_MODEL:
    # load model
    model = torch.load(LOAD_MODEL_PATH)
elif USE_BASE_MODEL:
    model = BASE_MODEL(pretrained = LOAD_PYTORCH_WEIGHTS)
    
    if BASE_MODEL_TRAINABLE:
        for param in model.parameters():
            param.requires_grad = False
    
    if INCLUDE_TOP:
        # Here the size of each output sample is set to 2.
        # Alternatively, it can be generalized to nn.Linear(num_ftrs, len(class_names))
        num_ftrs = model.fc.in_features
        model.fc = nn.Linear(num_ftrs, CLASSES) 
        
    # 載入完整模型架構與參數
    model = copy.deepcopy(model.state_dict())
else:
    # ==== INIT MODEL    
    model = build_model()

    if LOAD_WEIGHTS:
        # load model weights
        model.load_state_dict(LOAD_WEIGHTS_PATH)

model.to(DEVICE)
optimizer = optimizer(model.parameters(), lr = LEARNING_RATE, weight_decay = WEIGHT_DECAY)

# Later we have to filter the invalid steps.
criterion = loss(reduction = REDUCTION)

if MODEL_PRINT:
    # Print model's state_dict
    print("Model's state_dict:")
    for param_tensor in model.state_dict():
        print(param_tensor, "\t", model.state_dict()[param_tensor].size())

if OPTIMIZER_PRINT:
    # Print optimizer's state_dict
    print("Optimizer's state_dict:")
    for var_name in optimizer.state_dict():
        print(var_name, "\t", optimizer.state_dict()[var_name])

# 6. 製作資料集＆資料擴增&回調函數&訓練模型 <a class="anchor" id="6"></a>
[Back to Table of Contents](#0)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, df, transform = None):
        self.df = df
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx: int):
        label = self.df.target.values[idx]
        image_name = self.df.image_name.values[idx]
        image_path = TRAIN_DATA_PATH + image_name + IMAGE_NAME_EXTENSION
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label

In [ ]:
# 確定是否將應用此增強。機率為 p = 1.0 意味著我們總是從上面應用轉換。
# p = 0 將意味著將忽略轉換塊。
# 0 < p < 1.0 等於每個擴增都具有以一定概率應用的選項。
# OneOf 隨機選取一種增強擴增

def get_train_transforms():
    return A.Compose([
        A.Blur(blur_limit = BLUR_LIMIT, 
               p = P_BLUR), # 模糊
        A.HorizontalFlip(p = P_HORIZONTALFLIP), # 水平翻轉
        A.VerticalFlip(p = P_VERTICALFLIP), # 垂直翻轉
        A.Flip(p = P_FLIP), # 水平和垂直翻轉
        A.Resize(height = RESIZE_HEIGHT, 
                 width = RESIZE_WIDTH, 
                 p = P_RESIZE), # 縮放
        A.RandomResizedCrop(height = RANDOMRESIZEDCROP_HEIGHT, 
                            width = RANDOMRESIZEDCROP_WIDTH, 
                            scale = RANDOMRESIZEDCROP_SCALE, 
                            p = P_RANDOMRESIZEDCROP), #隨機縮放剪裁
        A.RandomRotate90(p = P_RANDOMROTATE90), # 隨機旋轉90度
        A.ShiftScaleRotate(shift_limit = SHIFTSCALEROTATE_SHIFT_LIMIT, 
                           scale_limit = SHIFTSCALEROTATE_SCALE_LIMIT, 
                           rotate_limit = SHIFTSCALEROTATE_ROTATE_LIMIT, 
                           p = P_SHIFTSCALEROTATE), # 平移縮放旋轉
        A.ElasticTransform(alpha = ELATICTRANSFORM_ALPHA, 
                           sigma = ELATICTRANSFORM_SIGMA, 
                           alpha_affine = ELATICTRANSFORM_ALPHA_AFFINE, 
                           p = P_ELATICTRANSFORM), # 彈性變換
        A.GridDistortion(num_steps = GRIDDISTORTION_NUM_STEPS, 
                         p = P_GRIDDISTORTION), # 網格失真
        A.RandomBrightnessContrast(brightness_limit = RANDOMBRIGHTNESSCONTRAST_BRIGHTNESS_LIMIT, 
                                   contrast_limit = RANDOMBRIGHTNESSCONTRAST_CONTRAST_LIMIT, 
                                   p = P_RANDOMBRIGHTNESSCONTRAST_CONTRAST), # 隨機亮度對比度
        A.HueSaturationValue(hue_shift_limit = HUESATURATIONVALUE_HUE_SHIFT_LIMIT, 
                             sat_shift_limit = HUESATURATIONVALUE_SAT_SHIFT_LIMIT, 
                             val_shift_limit = HUESATURATIONVALUE_VAL_SHIFT_LIMIT, 
                             p = P_HUESATURATIONVALUE), # 隨機色調飽和度值
        A.CLAHE(clip_limit = CLAHE_CLIP_LIMIT, 
                p = P_CLAHE), # 將對比度受限的自適應直方圖均衡化應用於輸入圖像
        A.CoarseDropout(num_holes = COARSEDROPOUT_NUM_HOLES, 
                        max_h_size = COARSEDROPOUT_MAX_H_SIZE, 
                        max_w_size = COARSEDROPOUT_MAX_W_SIZE, 
                        p = P_COARSEDROPOUT), # 隨機在圖像上生成黑色矩形
        A.Normalize(
             mean = NORMALIZE_MEAN, 
             std = NORMALIZE_STD, 
            max_pixel_value = NORMALIZE_MAX_PIXEL_VALUE, 
            p = P_NORMALIZE), # 正規化。
        ToTensorV2(p = P_TOTENSORV2) # 歸一化
    ], p = P_TRAIN_TRANSFORMS)

def get_val_transforms():
    return A.Compose([
        A.Resize(height = RESIZE_HEIGHT, 
                 width = RESIZE_WIDTH, 
                 p = P_RESIZE), # 縮放
        A.Normalize(
             mean = NORMALIZE_MEAN,
             std = NORMALIZE_STD, 
            max_pixel_value = NORMALIZE_MAX_PIXEL_VALUE, 
            p = P_NORMALIZE), # 正規化。
        ToTensorV2(p = P_TOTENSORV2) # 歸一化
    ], p = P_VAL_TRANSFORMS)

In [ ]:
#save the losses for further visualization
losses = {'train':[], 'val':[]}
accuracies = {'train':[], 'val':[]}

# Decay LR
scheduler = lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.3)

# 宣告為訓練後預測用
all_labels = []; all_pred = []

In [ ]:
def train_process(fold, skf, x_train, x_val, y_train, y_val):
    if skf:
        print('FOLD %i - IMAGE SIZE %i WITH %s AND BATCH_SIZE %i'%(fold+1,IMAGE_SIZE[fold],MODEL_NAME.upper(),BATCH_SIZE[fold]))
    else:
        print('IMAGE SIZE %i WITH %s AND BATCH_SIZE %i'%(IMAGE_SIZE[fold],MODEL_NAME.upper(),BATCH_SIZE[fold]))
        
    if LOAD_CSV:
        train_data = pd.DataFrame(x_train)
        train_data.columns = [IMAGE_NAME_ROOT]
        train_data[LABEL_NAME] = y_train

        validation_data = pd.DataFrame(x_val)
        validation_data.columns = [IMAGE_NAME_ROOT]
        validation_data[LABEL_NAME] = y_val

        train_data[LABEL_NAME] = train_data[LABEL_NAME].astype(LABEL_NAME_TYPE)
        validation_data[LABEL_NAME] = validation_data[LABEL_NAME].astype(LABEL_NAME_TYPE)
        
    train_set = MyDataset(train_data, transform = get_train_transforms())
    val_set = MyDataset(validation_data, transform = get_val_transforms())
    
    #for metrics
    dataset_sizes = { 'train': len(train_set), 'val': len(val_set)}
    print(dataset_sizes)
    
    loaders = {
        'train': DataLoader(train_set, batch_size = BATCH_SIZE[fold], 
                                               shuffle = True, num_workers = NUM_WORKERS, drop_last = DROP_LAST),
        'val': DataLoader(val_set, batch_size = BATCH_SIZE[fold], 
                                               shuffle = False, num_workers = NUM_WORKERS, drop_last = DROP_LAST)
    }
    
    val_acc = 0.0
    val_loss = 0.0
    val_auc = 0.0
    early_stopping = 0
    
    if LOAD_CSV and skf:
        # (String)訓練模型FOLD>1的儲存路徑
        SAVE_MODEL_PATH = PROJECT_PATH+r'/models/'+MODEL_NAME+'_fold_%i.pt'%(fold+1)
    else:
        SAVE_MODEL_PATH = TRAIN_MODEL_PATH
    
    since = time.time()
    for epoch in range(EPOCHS):  
        print('Epoch: {}/{}'.format(epoch, EPOCHS - 1))
        print('-' * 10)
        
        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train() # Set model to training mode
            else:
                model.eval() # Set model to evaluate mode
                
            running_loss = 0.0
            running_corrects = 0.0
            
            # 宣告為計算每次迭代auc
            valid_preds, valid_targets = [], []
                
            # Iterate over data.
            tq = tqdm(loaders[phase], total = len(loaders[phase]), position = TQDM_POSITION, leave = TQDM_LEAVE)
            for idx, data in enumerate(tq):
                # get the inputs; data is a list of [inputs, labels]
                inputs, labels = data[0].to(DEVICE), data[1].to(DEVICE)
                
                # zero the parameter gradients
                optimizer.zero_grad()
                
                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase=='train'):
                    outputs = model(inputs)
                    _, pred = torch.max(outp, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()
                    else:
                        valid_preds.append(torch.softmax(outputs,1)[:,1].detach().cpu().numpy())
                        valid_targets.append(labels.detach().cpu().numpy())
                   
                # statistics
                running_loss += loss.item()*inputs.size(0)
                running_corrects += torch.sum(pred == labels.data)
            
            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double()/dataset_sizes[phase]
            losses[phase].append(epoch_loss)
            accuracies[phase].append(epoch_acc)
            
            # 為了計算每次迭代auc
            valid_preds = np.concatenate(valid_preds)
            valid_targets = np.concatenate(valid_targets)
            epoch_auc =  roc_auc_score(valid_targets, valid_preds)
            
            # 為了計算全部迭代混淆矩陣
            all_pred = np.concatenate(valid_preds)
            all_labels = np.concatenate(valid_targets)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))
            
            if phase == 'val':
                print('Time: {}m {}s'.format((time.time()- since)//60, (time.time()- since)%60))
                
                if CALLBACKS_CHECK_POINTER:
                    save_model = False
                    if SAVE_BEST_ONLY:
                        if epoch_acc > val_acc and MONITOR == "val_acc":
                            val_acc = epoch_acc
                            save_model = True
                        elif epoch_loss < val_loss and MONITOR == "val_loss":
                            val_loss = epoch_loss
                            save_model = True
                        elif epoch_auc > val_auc and MONITOR == "val_auc":
                            val_auc = epoch_auc
                            save_model = True
                    else:
                        if MONITOR == "val_acc":
                            val_acc = epoch_acc
                        elif MONITOR == "val_loss":
                            val_loss = epoch_loss
                        elif MONITOR == "val_auc":
                            val_auc = epoch_auc
                        save_model = True
                        
                    if SAVE_WEIGHTS_ONLY and save_model:
                        torch.save(model.state_dict(), SAVE_MODEL_PATH)
                    elif not SAVE_WEIGHTS_ONLY and save_model:
                        torch.save(model, SAVE_MODEL_PATH)
                
#         scheduler.step()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    
    if SAVE_BEST_ONLY:
        if MONITOR == "val_acc":
            print('Best val Acc: {:4f}'.format(val_acc))
        elif MONITOR == "val_loss":
            print('Best val Loss: {:4f}'.format(val_loss))
        elif MONITOR == "val_auc":
            print('Best val AUC: {:4f}'.format(val_auc))
            
    if LOAD_CSV:
        del x_train, x_val, y_train, y_val
    del train_data, validation_data
    del loaders
    if LOAD_CSV and skf:
        del model
    gc.collect()

In [ ]:
# csv檔名加上副檔名，為了讀圖
if LOAD_CSV:
    label_list = []
    train_list = []
    for i in range(train_csv.shape[0]):
        train_list.append(TRAIN_DATA_PATH + '/' + train_csv[IMAGE_NAME].iloc[i] + IMAGE_NAME_EXTENSION)
        label_list.append(train_csv[LABEL_NAME].iloc[i])
    df_train = pd.DataFrame(train_list)
    df_train.columns = [IMAGE_NAME_ROOT]
    df_train[LABEL_NAME] = label_list

In [ ]:
if LOAD_CSV and FOLD == 1:
    X_train, X_val, Y_train, Y_val = train_test_split(df_train[IMAGE_NAME_ROOT],df_train[LABEL_NAME], test_size = DATA_SPLIT, random_state = SEED)
    train_process(fold = 0, skf = False, x_train = X_train, x_val = X_val, y_train = Y_train, y_val = Y_val)
    del X_train, X_val, Y_train, Y_val
    gc.collect()
elif LOAD_CSV and FOLD > 1:
    for fold,(train_index, val_index) in enumerate(SKF.split(df_train[IMAGE_NAME_ROOT], df_train[LABEL_NAME])):
        X_train, X_val = df_train[IMAGE_NAME_ROOT].iloc[train_index], df_train[IMAGE_NAME_ROOT].iloc[val_index]
        Y_train, Y_val = df_train[LABEL_NAME].iloc[train_index], df_train[LABEL_NAME].iloc[val_index]
        train_process(fold = fold, skf = True, x_train = X_train, x_val = X_val, y_train = Y_train, y_val = Y_val)
        del X_train, X_val, Y_train, Y_val
        gc.collect()
else:
    train_process(fold = 0, skf = False)

# 7. 混淆矩陣<a class="anchor" id="8"></a>
[Back to Table of Contents](#0)

# 8. 待辦事項<a class="anchor" id="9"></a>
[Back to Table of Contents](#0)

1. 製作資料集(無csv訓練)
2. 訓練(無csv訓練)
3. 回調函數(提早減少，學習率遞減，tensroboard)
4. 列印訓練過程
5. 混淆矩陣